<a href="https://colab.research.google.com/github/bhatmohit/FIFA_ScoutPro/blob/master/FIFA_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Model to predict most suitable playing positions for soccer players

## Load Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import re
# data = pd.read_csv('players_22.csv')

In [3]:
data = pd.read_excel('drive/MyDrive/FIFA_data_analysis/Players_Data.xlsx')

## EDA - Data Understanding

In [4]:
data.describe()

,Overall,Potential,Value(in Euro),Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Wage(in Euro),Release Clause,...,LM Rating,CM Rating,RM Rating,LWB Rating,CDM Rating,RWB Rating,LB Rating,CB Rating,RB Rating,GK Rating
count,18539.000000,18539.000000,1.853900e+04,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,1.853900e+04,...,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000,18539.000000
mean,65.852042,71.016668,2.875461e+06,25.240412,181.550839,75.173904,1602.114569,357.946221,8824.537462,5.081688e+06,...,58.451319,57.374076,58.451319,56.281569,55.928583,56.281569,55.650251,54.528184,55.650251,23.257134
std,6.788353,6.192866,7.635129e+06,4.718163,6.858097,7.013593,273.160237,39.628259,19460.531154,1.467203e+07,...,13.987122,13.171194,13.987122,13.903836,13.872190,13.903836,14.159466,14.743929,14.159466,15.108925
min,47.000000,48.000000,0.000000e+00,16.000000,155.000000,49.000000,759.000000,224.000000,0.000000,0.000000e+00,...,18.000000,18.000000,18.000000,17.000000,19.000000,17.000000,17.000000,18.000000,17.000000,10.000000
25%,62.000000,67.000000,4.750000e+05,21.000000,177.000000,70.000000,1470.000000,331.000000,1000.000000,6.650000e+05,...,54.000000,53.000000,54.000000,51.000000,48.000000,51.000000,49.000000,45.000000,49.000000,17.000000
50%,66.000000,71.000000,1.000000e+06,25.000000,182.000000,75.000000,1640.000000,358.000000,3000.000000,1.500000e+06,...,62.000000,60.000000,62.000000,59.000000,59.000000,59.000000,59.000000,58.000000,59.000000,18.000000
75%,70.000000,75.000000,2.000000e+06,29.000000,186.000000,80.000000,1786.000000,385.000000,8000.000000,3.400000e+06,...,67.000000,66.000000,67.000000,66.000000,66.000000,66.000000,65.000000,66.000000,65.000000,20.000000
max,91.000000,95.000000,1.905000e+08,44.000000,206.000000,105.000000,2312.000000,502.000000,450000.000000,3.667000e+08,...,92.000000,91.000000,92.000000,88.000000,89.000000,88.000000,87.000000,90.000000,87.000000,90.000000


In [5]:
data.columns.values
# data.select_dtypes(include=['int']).columns.values

array(['Known As', 'Full Name', 'Overall', 'Potential', 'Value(in Euro)',
       'Positions Played', 'Best Position', 'Nationality', 'Image Link',
       'Age', 'Height(in cm)', 'Weight(in kg)', 'TotalStats', 'BaseStats',
       'Club Name', 'Wage(in Euro)', 'Release Clause', 'Club Position',
       'Contract Until', 'Club Jersey Number', 'Joined On', 'On Loan',
       'Preferred Foot', 'Weak Foot Rating', 'Skill Moves',
       'International Reputation', 'National Team Name',
       'National Team Image Link', 'National Team Position',
       'National Team Jersey Number', 'Attacking Work Rate',
       'Defensive Work Rate', 'Pace Total', 'Shooting Total',
       'Passing Total', 'Dribbling Total', 'Defending Total',
       'Physicality Total', 'Crossing', 'Finishing', 'Heading Accuracy',
       'Short Passing', 'Volleys', 'Dribbling', 'Curve',
       'Freekick Accuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Shot Powe

## Data Preparation/Pre-Processing

In [6]:
# data = data.filter(['Full Name','Best Position', 'Pace Total', 'Shooting Total', 'Passing Total', 'Dribbling Total', 'Defending Total', 'Physicality Total', 'Crossing', 'Finishing', 'Short Passing', 'Vision', 'Standing Tackle', 'Sliding Tackle', 'Marking', 'Long Shots', 'Agility', 'Goalkeeper Diving', 'Goalkeeper Handling', ' GoalkeeperKicking'])
data = data.filter(['Full Name','Best Position', 'Pace Total', 'Shooting Total',
       'Passing Total', 'Dribbling Total', 'Defending Total',
       'Physicality Total', 'Crossing', 'Finishing', 'Heading Accuracy',
       'Short Passing', 'Volleys', 'Dribbling', 'Curve',
       'Freekick Accuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Shot Power',
       'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'Standing Tackle', 'Sliding Tackle',
       'Goalkeeper Diving', 'Goalkeeper Handling', ' GoalkeeperKicking',
       'Goalkeeper Positioning', 'Goalkeeper Reflexes', 'ST Rating',
       'LW Rating', 'LF Rating', 'CF Rating', 'RF Rating', 'RW Rating',
       'CAM Rating', 'LM Rating', 'CM Rating', 'RM Rating', 'LWB Rating',
       'CDM Rating', 'RWB Rating', 'LB Rating', 'CB Rating', 'RB Rating',
       'GK Rating'])
data

,Full Name,Best Position,Pace Total,Shooting Total,Passing Total,Dribbling Total,Defending Total,Physicality Total,Crossing,Finishing,...,LM Rating,CM Rating,RM Rating,LWB Rating,CDM Rating,RWB Rating,LB Rating,CB Rating,RB Rating,GK Rating
0,Lionel Messi,CAM,81,89,90,94,34,64,84,90,...,91,88,91,67,66,67,62,53,62,22
1,Karim Benzema,CF,80,88,83,87,39,78,75,92,...,89,84,89,67,67,67,63,58,63,21
2,Robert Lewandowski,ST,75,91,79,86,44,83,71,94,...,86,83,86,67,69,67,64,63,64,22
3,Kevin De Bruyne,CM,74,88,93,87,64,77,94,85,...,91,91,91,82,82,82,78,72,78,24
4,Kylian Mbapp√©,ST,97,89,80,92,36,76,78,93,...,92,84,92,70,66,70,66,57,66,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18534,Darren Collins,CAM,68,48,43,51,31,33,40,49,...,50,44,50,41,38,41,40,36,40,15
18535,Dejiang Yang,CDM,55,37,41,47,48,39,34,33,...,45,45,45,47,48,47,49,49,49,15
18536,Liam Mullan,RM,64,40,49,52,37,35,41,36,...,52,49,52,46,44,46,46,42,46,17
18537,Daith√≠ McCallion,CB,52,24,25,32,52,41,21,20,...,33,33,33,44,42,44,47,49,47,15


## Data Splitting

In [7]:
x = data.drop(['Best Position'], axis=1)
y = data['Best Position']

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 100)

In [9]:
x_train_names = x_train['Full Name']
x_train = x_train.drop('Full Name', axis=1)

x_test_names = x_test['Full Name']
x_test = x_test.drop('Full Name', axis=1)

In [10]:
x_test

,Pace Total,Shooting Total,Passing Total,Dribbling Total,Defending Total,Physicality Total,Crossing,Finishing,Heading Accuracy,Short Passing,...,LM Rating,CM Rating,RM Rating,LWB Rating,CDM Rating,RWB Rating,LB Rating,CB Rating,RB Rating,GK Rating
12022,71,63,47,56,36,75,35,65,59,57,...,58,55,58,50,50,50,49,50,49,18
13870,74,52,55,63,60,70,50,49,57,61,...,61,60,61,64,63,64,64,64,64,18
10641,65,58,61,66,61,61,45,51,58,70,...,65,67,65,65,66,65,65,64,65,18
11271,80,57,58,68,32,48,48,62,38,60,...,66,60,66,52,49,52,49,41,49,19
13925,73,29,46,56,58,55,56,22,58,56,...,54,50,54,61,56,61,62,60,62,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,89,57,73,78,74,73,78,45,68,77,...,78,76,78,80,77,80,80,76,80,20
9189,72,59,60,72,23,51,60,60,38,63,...,68,61,68,52,46,52,48,36,48,18
13913,75,42,56,60,59,60,60,39,42,62,...,59,56,59,62,60,62,62,60,62,15
8085,80,52,64,64,58,67,63,40,44,68,...,66,65,66,67,65,67,67,62,67,17


## Model Building

In [11]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(x_train, y_train)

KNeighborsClassifier()

In [12]:
# y_train_pred = neigh.predict(x_train)
knn_predictions = neigh.predict(x_test)
knn_predictions = pd.Series(knn_predictions)
knn_predictions

0        ST
1        LM
2       CAM
3       CAM
4        RB
       ... 
3703     LB
3704    CAM
3705     LB
3706    LWB
3707    CAM
Length: 3708, dtype: object

In [13]:
# y_test_pred
# y_test
# type(y_test)
knn_result = pd.concat([x_test_names.reset_index(drop=True), y_test.reset_index(drop=True), knn_predictions.reset_index(drop=True)], axis=1, ignore_index=True)
knn_result.columns = ['name', 'actual', 'predicted']
print(knn_result.head())


               name actual predicted
0   Fabian Schubert     ST        ST
1    Ovidiu Perianu    CDM        LM
2    Sami Lahssaini     CM       CAM
3    H√©ber Garc√≠a    CAM       CAM
4  Amankwaa Akurugu     LB        RB


66.72060409924487

In [21]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

random_forest.fit(x_train, y_train)

rf_predictions = random_forest.predict(x_test)

rf_predictions = pd.Series(rf_predictions)

rf_result = pd.concat([x_test_names.reset_index(drop=True), y_test.reset_index(drop=True), rf_predictions.reset_index(drop=True)], axis=1, ignore_index=True)
# rf_result = pd.concat([x_test_names, y_test, rf_predictions], axis=1, ignore_index=True)
rf_result.columns = ['name', 'actual', 'predicted']

## Evaluate Model Performance

In [22]:
#Import relevant libraries for evaluation

from sklearn.metrics import precision_score, recall_score, f1_score

In [23]:
#Evaluate KNN model performance

knn_result['check'] = np.where(knn_result['actual'] == knn_result['predicted'], 1, 0)
positives = len(knn_result[knn_result['check'] == 1])
knn_accuracy = (positives/knn_result.shape[0]) * 100

precision = precision_score(knn_result['actual'], knn_result['predicted'], average='weighted')
print('Precision: ', precision)

sensitivity = recall_score(knn_result['actual'], knn_result['predicted'], average='weighted')
print('Sensitivity: ', sensitivity)

knn_f1_score = f1_score(knn_result['actual'], knn_result['predicted'], average='weighted')
print('F1 Score: ', knn_f1_score)

# knn_result.dtypes

Precision:  0.6399257088443886
Sensitivity:  0.6672060409924487
F1 Score:  0.6472595948855357


In [24]:
#Evaluate Random Forest model performance
print(knn_result.dtypes)
print(rf_result.dtypes)

rf_result['check'] = np.where(rf_result['actual'] == rf_result['predicted'], 1, 0)
positives = len(rf_result[rf_result['check'] == 1])
rf_accuracy = (positives/rf_result.shape[0]) * 100

precision = precision_score(rf_result['actual'], rf_result['predicted'], average='weighted')
print('Precision: ', precision)

sensitivity = recall_score(rf_result['actual'], rf_result['predicted'], average='weighted')
print('Sensitivity: ', sensitivity)

rf_f1_score = f1_score(rf_result['actual'], rf_result['predicted'], average='weighted')
print('F1 Score: ', rf_f1_score)

name         object
actual       object
predicted    object
check         int64
dtype: object
name         object
actual       object
predicted    object
dtype: object
Precision:  0.6760290126375876
Sensitivity:  0.7173678532901834
F1 Score:  0.6868002511913089


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
